In [2]:
# risk_analysis_module.py

def calculate_risk_score(
    idade_empresa: float,
    cpf_score: int,
    divida_total: float,
    faturamento_anual: float,
    saldo_medio_diario: float,
    faturamento_medio_mensal: float,
    estresse_caixa_dias: int,
    valor_maior_cliente: float,
    faturamento_total_periodo: float,
    cnpj_score_api: int
) -> tuple[float, str]:
    """
    Calcula o score de risco e a classificação de uma empresa com base em suas métricas financeiras.
    Esta função é auto-contida e não depende de outros sistemas.
    """
    # --- 1. Normalização das Variáveis ---
    if idade_empresa < 2: risco_idade = 400
    elif idade_empresa < 5: risco_idade = 600
    else: risco_idade = 1000

    if cpf_score < 400: risco_cpf = 100
    elif cpf_score < 700: risco_cpf = 500
    elif cpf_score < 900: risco_cpf = 700
    else: risco_cpf = 1000

    if faturamento_anual > 0:
        indice_endividamento = divida_total / faturamento_anual
        if indice_endividamento > 1.0: risco_endividamento = 100
        elif indice_endividamento > 0.6: risco_endividamento = 300
        elif indice_endividamento > 0.3: risco_endividamento = 700
        else: risco_endividamento = 1000
    else:
        risco_endividamento = 0

    if faturamento_medio_mensal > 0:
        indice_liquidez = saldo_medio_diario / faturamento_medio_mensal
        if indice_liquidez < 0.1: risco_liquidez = 100
        elif indice_liquidez < 0.4: risco_liquidez = 500
        else: risco_liquidez = 1000
    else:
        risco_liquidez = 0

    if estresse_caixa_dias > 10: risco_estresse = 0
    elif estresse_caixa_dias > 5: risco_estresse = 300
    elif estresse_caixa_dias > 0: risco_estresse = 700
    else: risco_estresse = 1000

    if faturamento_total_periodo > 0:
        indice_concentracao = valor_maior_cliente / faturamento_total_periodo
        if indice_concentracao > 0.6: risco_concentracao = 100
        elif indice_concentracao > 0.3: risco_concentracao = 500
        elif indice_concentracao > 0.2: risco_concentracao = 900
        else: risco_concentracao = 1000
    else:
        risco_concentracao = 0

    # --- 2. Aplicação dos Pesos ---
    pesos = {
        "idade": 0.05, "cpf_score": 0.05, "endividamento": 0.15,
        "liquidez": 0.10, "estresse_caixa": 0.10, "concentracao": 0.05,
        "score_biro": 0.5,
    }
    risco_final = (
        risco_endividamento * pesos["endividamento"] + risco_liquidez * pesos["liquidez"] +
        risco_estresse * pesos["estresse_caixa"] + risco_concentracao * pesos["concentracao"] +
        risco_idade * pesos["idade"] + risco_cpf * pesos["cpf_score"] +
        cnpj_score_api * pesos["score_biro"]
    )
    risco_final = max(0, min(1000, risco_final))

    # --- 3. Classificação do Risco ---
    if risco_final > 800: classificacao = 'A'
    elif risco_final > 600: classificacao = 'B'
    elif risco_final > 400: classificacao = 'C'
    elif risco_final > 200: classificacao = 'D'
    else: classificacao = 'automatically_reproved'
    
    return round(risco_final, 2), classificacao

def calcular_taxa_juros_anual(
    risco_final: float,
    prazo_meses: int,
    valor_solicitado: float
) -> float:
    """
    Calcula a taxa de juros anual com base no risco, prazo e valor.
    """
    TAXA_BASE = 0.12
    premio_risco = (-(risco_final - 1000) / 1000) * 0.25
    
    if prazo_meses <= 6: premio_prazo = 0.00
    elif prazo_meses <= 12: premio_prazo = 0.02
    elif prazo_meses <= 18: premio_prazo = 0.04
    else: premio_prazo = 0.06

    if valor_solicitado < 50000: premio_valor = 0.00
    elif valor_solicitado < 150000: premio_valor = 0.01
    elif valor_solicitado < 300000: premio_valor = 0.025
    else: premio_valor = 0.05
        
    taxa_final_anual = TAXA_BASE + (premio_risco + premio_prazo + premio_valor)
    return taxa_final_anual